In [1]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import numpy as np
from datetime import timedelta, datetime
from maritime_traffic_network import MaritimeTrafficNetwork
import folium
import warnings
import sys

warnings.filterwarnings('ignore')

print("Geopandas has version {}".format(gpd.__version__))
print("Movingpandas has version {}".format(mpd.__version__))

Geopandas has version 0.13.2
Movingpandas has version 0.17.1


In [2]:
# add paths for modules
sys.path.append('../visualization')
print(sys.path)

# import modules
import visualize

['/Users/janhendrikwebert/maritime_route_prediction/src/models', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python311.zip', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11/lib-dynload', '', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11/site-packages', '../visualization']


In [3]:
# read data from file
filename = '../../data/processed/202204_points_stavanger_cleaned_meta_2M.parquet'
#filename = '../../data/processed/202204_points_stavanger_cleaned_full.parquet'
gdf = gpd.read_parquet(filename)
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
DatetimeIndex: 1879232 entries, 2022-04-08 03:12:48 to 2022-04-08 23:59:59
Data columns (total 17 columns):
 #   Column        Dtype   
---  ------        -----   
 0   mmsi          object  
 1   imo_nr        int64   
 2   length        int64   
 3   lon           float64 
 4   lat           float64 
 5   sog           float64 
 6   cog           float64 
 7   true_heading  int64   
 8   nav_status    int64   
 9   message_nr    int64   
 10  bredde        float64 
 11  dypgaaende    float64 
 12  skipstype     object  
 13  skipsgruppe   object  
 14  fartoynavn    object  
 15  geometry      geometry
 16  speed         float64 
dtypes: float64(7), geometry(1), int64(5), object(4)
memory usage: 258.1+ MB


In [4]:
# initialize maritime traffic network
network = MaritimeTrafficNetwork(gdf)
network.get_trajectories_info()

AIS messages: 1879232
Trajectories: 2674


In [5]:
# calculate significant turning points using Douglas Peucker algorithm
tolerance = 0.002  # DP tolerance parameter
network.calc_significant_points_DP(tolerance)

Calculating significant turning points with Douglas Peucker algorithm (tolerance = 0.002) ...
Number of significant points detected: 29963 (1.59% of AIS messages)
Time elapsed: 1.12 minutes
Adding course over ground before and after each turn ...
Done. Time elapsed: 1.43 minutes


In [6]:
# select trajectories
trajs = network.trajectories.to_traj_gdf()
simplified_trajs = network.significant_points_trajectory.to_point_gdf()

In [7]:
from shapely.geometry import LineString, Point
from math import atan2, cos, degrees, pi, radians, sin, sqrt
def calculate_initial_compass_bearing(point1, point2):
    """
    Calculate the bearing between two points.

    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `point1: shapely Point
      - `point2: shapely Point
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    lat1 = radians(point1.y)
    lat2 = radians(point2.y)
    delta_lon = radians(point2.x - point1.x)
    x = sin(delta_lon) * cos(lat2)
    y = cos(lat1) * sin(lat2) - (sin(lat1) * cos(lat2) * cos(delta_lon))
    initial_bearing = atan2(x, y)
    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

In [9]:
import time
start = time.time()  # start timer
simplified_trajs['cog_before'] = np.nan
simplified_trajs['cog_after'] = np.nan
for mmsi in simplified_trajs.mmsi.unique():
    subset = simplified_trajs[simplified_trajs.mmsi == mmsi]
    if len(subset)>1:
        for i in range(0, len(subset)-1):
            subset['cog_before'].iloc[i+1] = (calculate_initial_compass_bearing(subset['geometry'].iloc[i], subset['geometry'].iloc[i+1]))
            subset['cog_after'].iloc[i] = subset['cog_before'].iloc[i+1]
        subset['cog_before'].iloc[0] = subset['cog_before'].iloc[1]
        subset['cog_after'].iloc[-1] = subset['cog_after'].iloc[-2]
    else:
        subset['cog_after'] = 0
        subset['cog_before'] = 0
    simplified_trajs.update(subset)
end = time.time()  # end timer
print(f'Done. Time elapsed: {(end-start)/60:.2f} minutes')

Done. Time elapsed: 1.44 minutes


In [ ]:
mmsi = '219347000_0'
trajs = trajs[trajs.mmsi == mmsi]
simplified_trajs = simplified_trajs[simplified_trajs.mmsi == mmsi]
simplified_trajs.reset_index(inplace=True)

In [ ]:
simplified_trajs

In [ ]:
sig = network.significant_points
sig = sig[sig.mmsi == mmsi]
sig.reset_index(inplace=True)

In [ ]:
columns = ['geometry', 'mmsi', 'cog_before', 'cog_after']  # columns to be plotted
# plot simplified trajectories
#map = trajs[['geometry', 'mmsi']].explore(column='mmsi', name='accurate trajectories', 
#                             style_kwds={'weight':2, 'color':'black', 'opacity':0.5}, 
#                             legend=False)
# plot significant turning points with their cluster ID

map = simplified_trajs[columns].explore(name='simplified trajectories', legend=False,
                                        style_kwds={'weight':2, 'color':'blue', 'opacity':0.5})

map = sig[columns].explore(m=map, name='simplified trajectories2', legend=False,
                                                            style_kwds={'weight':2, 'color':'red', 'opacity':0.5})


folium.LayerControl().add_to(map)
map